In [1]:
from openfermion.hamiltonians import MolecularData
from openfermion.transforms import get_fermion_operator, get_sparse_operator, jordan_wigner,bravyi_kitaev
from openfermion.utils import get_ground_state,eigenspectrum,count_qubits
import numpy as np
import scipy
import scipy.linalg
from openfermionpyscf import run_pyscf

In [5]:
element_names = ['H', 'Li']
basis = 'sto-6g'
charge = 0
multiplicity = 1

# Single point at equilibrium for testing
spacings = [1.6]

# Add points for a full dissociation curve from 0.1 to 3.0 angstroms
#spacings += [0.2 * r for r in range(1, 11)]

# Set run options
run_scf = 1
run_mp2 = 1
run_cisd = 1
run_ccsd = 1
run_fci = 1
verbose = 1

# Run Diatomic Curve
for spacing in spacings:
    description = "{}".format(spacing)
    geometry = [[element_names[0], [0, 0, 0]],
                [element_names[1], [0, 0, spacing]]]
    molecule = MolecularData(geometry,
                             basis,
                             multiplicity,
                             charge,
                             description)

    molecule = run_pyscf(molecule,
                         run_scf=run_scf,
                         run_mp2=run_mp2,
                         run_cisd=run_cisd,
                         run_ccsd=run_ccsd,
                         run_fci=run_fci,
                         verbose=verbose)
    molecule.save()

Hartree-Fock energy for H1-Li1_sto-6g_singlet_1.6 (4 electrons) is -7.951804963446999.
MP2 energy for H1-Li1_sto-6g_singlet_1.6 (4 electrons) is -7.964714755083031.
CISD energy for H1-Li1_sto-6g_singlet_1.6 (4 electrons) is -7.972236911480958.
CCSD energy for H1-Li1_sto-6g_singlet_1.6 (4 electrons) is -7.9722398782146735.
FCI energy for H1-Li1_sto-6g_singlet_1.6 (4 electrons) is -7.972249851370502.


# the total Hamiltonian. Require 12 qubits

In [6]:
molecular_hamiltonian = molecule.get_molecular_hamiltonian()

In [9]:
molecular_hamiltonian

() 0.992207270475
((0, 1), (0, 0)) -4.773344324649143
((0, 1), (2, 0)) 0.10451979828068142
((0, 1), (4, 0)) 0.1693113159088834
((0, 1), (10, 0)) -0.040397591518086154
((1, 1), (1, 0)) -4.773344324649143
((1, 1), (3, 0)) 0.10451979828068142
((1, 1), (5, 0)) 0.1693113159088834
((1, 1), (11, 0)) -0.040397591518086154
((2, 1), (0, 0)) 0.10451979828068131
((2, 1), (2, 0)) -1.4974610947637153
((2, 1), (4, 0)) 0.028954809876305686
((2, 1), (10, 0)) -0.05050127678786433
((3, 1), (1, 0)) 0.10451979828068131
((3, 1), (3, 0)) -1.4974610947637153
((3, 1), (5, 0)) 0.028954809876305686
((3, 1), (11, 0)) -0.05050127678786433
((4, 1), (0, 0)) 0.1693113159088832
((4, 1), (2, 0)) 0.028954809876305734
((4, 1), (4, 0)) -1.1313065946179366
((4, 1), (10, 0)) 0.034927275776870846
((5, 1), (1, 0)) 0.1693113159088832
((5, 1), (3, 0)) 0.028954809876305734
((5, 1), (5, 0)) -1.1313065946179366
((5, 1), (11, 0)) 0.034927275776870846
((6, 1), (6, 0)) -1.138263611027671
((7, 1), (7, 0)) -1.138263611027671
((8, 1), (

In [10]:
# Map operator to fermions and qubits.
fermion_hamiltonian = get_fermion_operator(molecular_hamiltonian)

In [11]:
fermion_hamiltonian

0.992207270475 [] +
-4.773344324649143 [0^ 0] +
0.8322582015735619 [0^ 0^ 0 0] +
-0.05540604485854806 [0^ 0^ 0 2] +
-0.07074385330443898 [0^ 0^ 0 4] +
0.029329016914407896 [0^ 0^ 0 10] +
-0.055406044858548074 [0^ 0^ 2 0] +
0.006685117037753123 [0^ 0^ 2 2] +
0.005787266405732589 [0^ 0^ 2 4] +
-0.004833410326303781 [0^ 0^ 2 10] +
-0.07074385330443901 [0^ 0^ 4 0] +
0.005787266405732588 [0^ 0^ 4 2] +
0.011426770562088111 [0^ 0^ 4 4] +
-0.0016704427711028158 [0^ 0^ 4 10] +
0.005027121617449969 [0^ 0^ 6 6] +
0.005027121617449969 [0^ 0^ 8 8] +
0.029329016914407965 [0^ 0^ 10 0] +
-0.004833410326303789 [0^ 0^ 10 2] +
-0.001670442771102827 [0^ 0^ 10 4] +
0.004712930152978252 [0^ 0^ 10 10] +
0.8322582015735619 [0^ 1^ 1 0] +
-0.05540604485854806 [0^ 1^ 1 2] +
-0.07074385330443898 [0^ 1^ 1 4] +
0.029329016914407896 [0^ 1^ 1 10] +
-0.055406044858548074 [0^ 1^ 3 0] +
0.006685117037753123 [0^ 1^ 3 2] +
0.005787266405732589 [0^ 1^ 3 4] +
-0.004833410326303781 [0^ 1^ 3 10] +
-0.07074385330443901 [0^ 1^ 

In [13]:
qubit_hamiltonian = bravyi_kitaev(fermion_hamiltonian)
qubit_hamiltonian.compress()
print('The bravyi_kitaev Hamiltonian in canonical basis follows:\n{}'.format(qubit_hamiltonian))

The bravyi_kitaev Hamiltonian in canonical basis follows:
-4.190675029266606 [] +
0.027703022429274034 [X0 X1 X2] +
-0.001163866874013841 [X0 X1 X2 X3 Y7 Y11] +
0.000834753528520669 [X0 X1 X2 Y3 Y5] +
0.001573072965882944 [X0 X1 X2 Z3] +
0.0028644887276281106 [X0 X1 Z2 X3 Y7 Z9 Y10 X11] +
0.0030605743656053445 [X0 X1 Z2 Y3 Y4 X5] +
-0.0011486340643386502 [X0 X1 X3 X4 Y7 Y11] +
0.0011780425004470111 [X0 X1 X3 Y4 Y5 Z6 Z7] +
0.0011486340643386502 [X0 X1 X3 Y4 Z5 Y7 Z9 Z10 X11] +
0.002558384366176549 [X0 X1 X3 Z4 Y5 Y6 Z7] +
-0.002663691154223006 [X0 X1 X3 Z4 Z5 Y7 Z9 Y10 X11] +
-0.0015150570898843558 [X0 X1 X3 Z4 Y7 Z9 Y10 X11] +
0.001555214874273017 [X0 X1 X3 X6 Y7 Y11] +
-0.0017011005660201764 [X0 X1 X3 Z6 Y7 Z9 Y10 X11] +
0.001555214874273017 [X0 X1 X3 Y7 X8 Y11] +
-0.001555214874273017 [X0 X1 X3 Y7 Y8 Z10 X11] +
-0.0017011005660201764 [X0 X1 X3 Y7 Z8 Z9 Y10 X11] +
-0.00014588569174715934 [X0 X1 X3 Y7 Z8 Y10 X11] +
-0.0030161474425575915 [X0 X1 X3 Y7 Z9 Y10 X11] +
0.000789470319118171

# active space on 1,2,3 orbits. Only 6 qubits required (considering spin degree of freedom)

In [14]:
active_space_start = 1
active_space_stop = 4

molecular_hamiltonian = molecule.get_molecular_hamiltonian(
    occupied_indices=[0],
   active_indices=range(active_space_start, active_space_stop))

In [15]:
# Map operator to fermions and qubits.
fermion_hamiltonian = get_fermion_operator(molecular_hamiltonian)
qubit_hamiltonian = bravyi_kitaev(fermion_hamiltonian)
qubit_hamiltonian.compress()
print('The bravyi_kitaev Hamiltonian in canonical basis follows:\n{}'.format(qubit_hamiltonian))

The bravyi_kitaev Hamiltonian in canonical basis follows:
-7.352420783183575 [] +
0.011766123640336485 [X0 X1 X2] +
0.0018147425194416436 [X0 X1 X2 Z3] +
0.00475774066785974 [X0 X1 Z3 X4] +
0.0116834087535246 [X0 Y1 Y2] +
0.0033276028910050803 [X0 Y1 Y2 Z4] +
-0.0014301377768546593 [X0 Y1 Y2 Z4 Z5] +
0.00475774066785974 [X0 Y1 Z2 Y4 Z5] +
0.0030560036156448357 [X0 Z1 X2] +
0.0030560036156448357 [X0 Z1 X2 Z3] +
0.00575419931365656 [X0 Z1 X4] +
0.00575419931365656 [X0 X4 Z5] +
0.011766123640336485 [Y0 X1 Y2] +
0.0018147425194416436 [Y0 X1 Y2 Z3] +
0.00475774066785974 [Y0 X1 Z3 Y4] +
-0.0116834087535246 [Y0 Y1 X2] +
-0.0033276028910050803 [Y0 Y1 X2 Z4] +
0.0014301377768546593 [Y0 Y1 X2 Z4 Z5] +
-0.00475774066785974 [Y0 Y1 Z2 X4 Z5] +
0.0030560036156448357 [Y0 Z1 Y2] +
0.0030560036156448357 [Y0 Z1 Y2 Z3] +
0.00575419931365656 [Y0 Z1 Y4] +
0.00575419931365656 [Y0 Y4 Z5] +
0.027882756551383517 [Z0] +
0.011766123640336485 [Z0 X1 Z2] +
0.0018147425194416438 [Z0 X1 Z2 Z3] +
-0.01168340875352460

# Construct effective Hamiltonian on qubits 0,2,4, by average on |1> for qubit 1,3,5

> using method in “Quantum chemistry calculations on a trapped-ion quantum simulator” ,Physical Review X 8, 031022 (2018)

In [16]:
terms_dict=qubit_hamiltonian.terms

In [17]:
# the first qubit is 
def partial_average(key,drop_qubits=[1,3,5],avg_dict={'X':0,'Y':0,'Z':1},init_fock=0):
    key=list(key)
    factor=1
    new_key=[]
    for k in key:
        if k[0] not in drop_qubits:
            new_key.append(k)
        if k[0] in drop_qubits:
            if k[0]==0:
                factor*=avg_dict[k[1]]
            else:
                factor*=avg_dict[k[1]]
        #print(key)
    new_key=tuple(new_key)
    {new_key:factor}
    return (new_key,factor)

In [18]:
reduced_terms=[]
for key in terms_dict.keys():
    rt=partial_average(key)   
    reduced_terms.append(rt)
reduced_terms

[((), 1),
 (((0, 'Z'),), 1),
 (((0, 'X'), (2, 'Y')), 0),
 (((0, 'Y'), (2, 'X')), 0),
 (((0, 'Z'),), 1),
 (((0, 'Z'),), 0),
 (((2, 'Z'),), 0),
 (((2, 'Z'),), 1),
 (((2, 'Z'),), 1),
 (((4, 'Z'),), 1),
 (((4, 'Z'),), 1),
 ((), 1),
 (((0, 'Y'), (2, 'Y')), 0),
 (((0, 'X'), (2, 'X')), 0),
 ((), 0),
 (((0, 'Z'), (2, 'Z')), 0),
 (((0, 'Y'), (2, 'Y')), 1),
 (((0, 'X'), (2, 'X')), 1),
 (((0, 'X'), (2, 'X')), 1),
 (((0, 'Y'), (2, 'Y')), 1),
 (((0, 'Y'), (4, 'Y')), 1),
 (((0, 'X'), (4, 'X')), 1),
 (((0, 'X'), (4, 'X')), 1),
 (((0, 'Y'), (4, 'Y')), 1),
 (((0, 'Z'), (2, 'Z')), 1),
 (((0, 'Z'), (2, 'Z')), 1),
 (((0, 'X'), (2, 'X')), 0),
 (((0, 'Y'), (2, 'Y')), 0),
 (((0, 'X'), (2, 'Z'), (4, 'Y')), 0),
 (((0, 'X'), (4, 'X')), 0),
 (((0, 'Y'), (2, 'Z'), (4, 'X')), 0),
 (((0, 'Y'), (4, 'Y')), 0),
 (((0, 'Z'), (4, 'Z')), 1),
 (((0, 'X'), (2, 'Y'), (4, 'Z')), 0),
 (((0, 'Y'), (2, 'X'), (4, 'Z')), 0),
 (((0, 'Z'), (4, 'Z')), 1),
 (((0, 'X'), (2, 'Y'), (4, 'Z')), 0),
 (((0, 'Y'), (2, 'X'), (4, 'Z')), 0),
 (

## combining all same terms in sim_dict

In [19]:
import numpy as np
ham_terms=np.array([f[0] for f in reduced_terms])
factors=np.array([f[1] for f in reduced_terms])
cs=np.array([c for c in qubit_hamiltonian.terms.values()])
cs_rescale=np.multiply(factors,cs)

reduced_terms_rescale=[]
for i in range(len(reduced_terms)):
    if cs_rescale[i] !=0:
        reduced_terms_rescale.append((reduced_terms[i][0],cs_rescale[i]))
reduced_terms_rescale

[((), -7.352420783183575),
 (((0, 'Z'),), 0.027882756551383517),
 (((0, 'Z'),), 0.027882756551383545),
 (((2, 'Z'),), -0.1467285465541516),
 (((2, 'Z'),), -0.14672854655415157),
 (((4, 'Z'),), -0.16125794620832365),
 (((4, 'Z'),), -0.16125794620832368),
 ((), 0.12254342100191959),
 (((0, 'Y'), (2, 'Y')), 0.0030560036156448357),
 (((0, 'X'), (2, 'X')), 0.0030560036156448357),
 (((0, 'X'), (2, 'X')), 0.0030560036156448357),
 (((0, 'Y'), (2, 'Y')), 0.0030560036156448357),
 (((0, 'Y'), (4, 'Y')), 0.00575419931365656),
 (((0, 'X'), (4, 'X')), 0.00575419931365656),
 (((0, 'X'), (4, 'X')), 0.00575419931365656),
 (((0, 'Y'), (4, 'Y')), 0.00575419931365656),
 (((0, 'Z'), (2, 'Z')), 0.052625971801706126),
 (((0, 'Z'), (2, 'Z')), 0.05568197541735097),
 (((0, 'Z'), (4, 'Z')), 0.06175754692930635),
 (((0, 'Z'), (4, 'Z')), 0.06751174624296291),
 (((0, 'Z'), (2, 'Z')), 0.05568197541735097),
 (((0, 'Z'), (2, 'Z')), 0.052625971801706126),
 (((0, 'Z'), (4, 'Z')), 0.06751174624296291),
 (((0, 'Z'), (4, '

In [20]:
sim_dict={}
for term in reduced_terms_rescale:
    if term not in sim_dict.keys():
        sim_dict[term[0]]=term[1]
    else:
        sim_dict[term[0]]+=term[1]
[sim_dict.values()]

[dict_values([0.0782811414931556, 0.027882756551383545, -0.14672854655415157, -0.16125794620832368, 0.0030560036156448357, 0.0030560036156448357, 0.00575419931365656, 0.00575419931365656, 0.052625971801706126, 0.06175754692930635, 0.010350244330065435, 0.010350244330065435, 0.060282913124721615])]